In [190]:
import findspark
findspark.init()

from pyspark.sql import *
from pyspark.sql.types import *

from matplotlib import pyplot as plt
from plotly import graph_objects as go

from scapy.all import *
import os

In [205]:
spark = SparkSession.builder.appName('main').getOrCreate()
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
sparkContext = spark.sparkContext


In [195]:
main_folder = "C:\\Users\\igor.beryozkin\\Desktop\\docs\\PCAP"

files_list = []
for file in os.listdir(main_folder):
    pcap_folder = 'C:\\Users\\igor.beryozkin\\Desktop\\docs\\PCAP\\{0}'.format(file)
    files_list.append(pcap_folder)

In [196]:
files_list

['C:\\Users\\igor.beryozkin\\Desktop\\docs\\PCAP\\1_pack.pcap',
 'C:\\Users\\igor.beryozkin\\Desktop\\docs\\PCAP\\2_pack.pcap',
 'C:\\Users\\igor.beryozkin\\Desktop\\docs\\PCAP\\3_pack.pcap',
 'C:\\Users\\igor.beryozkin\\Desktop\\docs\\PCAP\\4_pack.pcap',
 'C:\\Users\\igor.beryozkin\\Desktop\\docs\\PCAP\\5_pack.pcap',
 'C:\\Users\\igor.beryozkin\\Desktop\\docs\\PCAP\\6_pack.pcap',
 'C:\\Users\\igor.beryozkin\\Desktop\\docs\\PCAP\\7_pack.pcap',
 'C:\\Users\\igor.beryozkin\\Desktop\\docs\\PCAP\\8_pack.pcap',
 'C:\\Users\\igor.beryozkin\\Desktop\\docs\\PCAP\\9_pack.pcap']

In [197]:
#Таблица признаков и их описание
F_columns = ['№', 'Признак', 'Описание']
data = [('1','adsada','asdasdasdassd')]
F_data = [(1, 'Размер запроса в байтах', 'Отклонение от средних показателей 40-60 байт в сторону увеличения может говорить о скрытом канале'),
          (2, 'Необычный тип трафика', 'Если в трафике используются данные вида TXT, NULL, KEY - это говорит об аномалии'),
          (3, 'Длина имени запроса', 'Имена используемые злоумышленниками могут достигать 200 символов - средняя 30 символов')]

In [198]:
feauture_DF = spark.createDataFrame(F_data, F_columns)

In [199]:
feauture_DF.show(truncate= 0)

+---+-----------------------+-------------------------------------------------------------------------------------------------+
|№  |Признак                |Описание                                                                                         |
+---+-----------------------+-------------------------------------------------------------------------------------------------+
|1  |Размер запроса в байтах|Отклонение от средних показателей 40-60 байт в сторону увеличения может говорить о скрытом канале|
|2  |Необычный тип трафика  |Если в трафике используются данные вида TXT, NULL, KEY - это говорит об аномалии                 |
|3  |Длина имени запроса    |Имена используемые злоумышленниками могут достигать 200 символов - средняя 30 символов           |
+---+-----------------------+-------------------------------------------------------------------------------------------------+



In [200]:
#DNS TUNNELING PCAP FILE FOR EXAMPLE 

#ioline = 'C://Users//igor.beryozkin//Downloads/dns-tunnel-iodine.pcap'
#a = rdpcap(ioline)

In [210]:
def ExtractData (file):
    extracted_data = []
    read_file = rdpcap(file)
    
    for pkt in read_file:

        if pkt.haslayer(DNSQR):
            query = pkt.getlayer(DNSQR)

            qtype_field = query.get_field('qtype')
            qtype_data = qtype_field.i2repr(query, query.qtype)

            qname_field = query.get_field('qname')
            qname_data = qname_field.i2repr(query, query.qname)
            
            extracted_data.append((qtype_data, qname_data.__len__(), pkt.len)) #TODO: rdd for optimizing processing
            
    return extracted_data #---> [(...), (...), ...]

In [211]:
#TODO: recreate func, use rdd for create dataframe, instead of list's

def CreateDataFrame (filesList, dfSchema):
    main_DF = spark.createDataFrame([], dfSchema) 
    
    for file in filesList:
        rdd = sparkContext.parallelize(ExtractData(file))
        
    return main_DF


In [212]:
df_schema = StructType([StructField('TrafficType', StringType(), True),
                             StructField('NameLenght', IntegerType(), True),
                             StructField('Bytes', IntegerType(), True)])

TypeError: 'NoneType' object is not iterable

In [ ]:
#collected_data_df = CreateDataFrame(files_list, df_schema)
#writed_df_parquet = collected_data_df.write.parquet('C:\\Users\\igor.beryozkin\\Desktop\\docs\\PARQUET')

In [ ]:
#RDD-create example.

# data = sparkContext.parallelize(ExtractData(files_list[1]))
# dfdf = spark.createDataFrame(data, df_schema)